In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

%matplotlib inline
init_notebook_mode(connected=True)

In [12]:
bechdel_df = pd.read_csv('my_data/updated_bechdel_4.csv')
bechdel_df.head()

,year,title,score,passing,imdb_id,director_gender,writer_gender,exec_gender,producer_gender,overall_gender,budget,revenue
0,2019.0,Alita: Battle Angel,3.0,1.0,tt0437086,0.0,1.0,0.0,0.0,1.0,170000000.0,359700000.0
1,2019.0,Close,3.0,1.0,tt5316540,1.0,1.0,0.0,0.0,2.0,0.0,0.0
2,2019.0,Cold Pursuit,1.0,0.0,tt5719748,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019.0,The Kid Who Would Be King,1.0,0.0,tt6811018,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,2019.0,Polar,2.0,0.0,tt4139588,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## I. Explore overall comparisons of passing vs non-passing movies

### A. Budget & Revenue

In [13]:
bechdel_df.groupby('passing').mean()

,year,score,director_gender,writer_gender,exec_gender,producer_gender,overall_gender,budget,revenue
passing,,,,,,,,,
0.0,1991.026761,1.000615,0.035681,0.083359,0.015995,0.123962,0.258997,2.116392e+07,6.638363e+07
1.0,1998.382940,3.000000,0.141252,0.224719,0.026141,0.174272,0.566155,1.657806e+07,5.787114e+07


Observation: budget and revenue are not easily interpretable as such large numbers. Scaling and using only non-zero values to determine the mean would help.

#### Non-zero values only

In [14]:
budget_and_revenue_df = bechdel_df[(bechdel_df.budget != 0) & (bechdel_df.revenue != 0)]
budget_and_revenue_df.head()

,year,title,score,passing,imdb_id,director_gender,writer_gender,exec_gender,producer_gender,overall_gender,budget,revenue
0,2019.0,Alita: Battle Angel,3.0,1.0,tt0437086,0.0,1.0,0.0,0.0,1.0,170000000.0,359700000.0
8,2018.0,A Quiet Place,1.0,0.0,tt6644200,0.0,0.0,0.0,0.0,0.0,17000000.0,259168321.0
9,2018.0,A Quiet Place,1.0,0.0,tt6644200,0.0,0.0,0.0,0.0,0.0,17000000.0,340939361.0
10,2018.0,A Simple Favor,3.0,1.0,tt7040874,0.0,1.0,0.0,1.0,2.0,20000000.0,42600000.0
11,2018.0,A Star Is Born,3.0,1.0,tt1517451,0.0,1.0,0.0,0.0,1.0,36000000.0,403588149.0


#### Scale budget and revenue values to millions

In [15]:
budget_and_revenue_df.budget = budget_and_revenue_df.budget / 1e6
budget_and_revenue_df.head()

,year,title,score,passing,imdb_id,director_gender,writer_gender,exec_gender,producer_gender,overall_gender,budget,revenue
0,2019.0,Alita: Battle Angel,3.0,1.0,tt0437086,0.0,1.0,0.0,0.0,1.0,170.0,359700000.0
8,2018.0,A Quiet Place,1.0,0.0,tt6644200,0.0,0.0,0.0,0.0,0.0,17.0,259168321.0
9,2018.0,A Quiet Place,1.0,0.0,tt6644200,0.0,0.0,0.0,0.0,0.0,17.0,340939361.0
10,2018.0,A Simple Favor,3.0,1.0,tt7040874,0.0,1.0,0.0,1.0,2.0,20.0,42600000.0
11,2018.0,A Star Is Born,3.0,1.0,tt1517451,0.0,1.0,0.0,0.0,1.0,36.0,403588149.0


In [16]:
budget_and_revenue_df.revenue = budget_and_revenue_df.revenue / 1e6
budget_and_revenue_df.rename(index=str, columns={'budget': 'budget_millions', 'revenue': 'revenue_millions'}, inplace=True)

In [17]:
budget_and_revenue_df.tail()

,year,title,score,passing,imdb_id,director_gender,writer_gender,exec_gender,producer_gender,overall_gender,budget_millions,revenue_millions
7570,1959.0,Sleeping Beauty,3.0,1.0,tt0053285,0.0,0.0,0.0,0.0,0.0,6.0,51.6
7588,2008.0,The Tale of Despereaux,3.0,1.0,tt0420238,0.0,0.0,0.0,1.0,1.0,60.0,86.9
7589,2007.0,Rush Hour 3,1.0,0.0,tt00293564,0.0,0.0,0.0,0.0,0.0,140.0,258.0
7592,1999.0,Pokemon: The Movie 2000,3.0,1.0,tt0210234,0.0,0.0,0.0,1.0,1.0,30.0,133.9
7609,2013.0,The Immigrant,3.0,1.0,tt1951181,0.0,0.0,0.0,0.0,0.0,16.0,5.9


In [18]:
budget_and_revenue_df.to_csv('my_data/non_zero_scaled_budgets_revenues.csv', index=False)

In [21]:
budget_and_revenue_df.groupby('passing').budget_millions.agg(['min', 'max', 'sum', 'mean', 'median'])

,min,max,sum,mean,median
passing,,,,,
0.0,0.000001,350.0,66011.138628,41.386294,25.0
1.0,0.000003,400.0,69031.694579,36.275194,20.0


#### Movies with minimum and maximum budgets

In [29]:
budget_and_revenue_df.loc[budget_and_revenue_df.budget_millions == budget_and_revenue_df.budget_millions.min()]

,year,title,score,passing,imdb_id,director_gender,writer_gender,exec_gender,producer_gender,overall_gender,budget_millions,revenue_millions
7309,1935.0,A Night At The Opera,1.0,0.0,tt0026778,0.0,0.0,0.0,0.0,0.0,0.000001,0.000001


In [30]:
budget_and_revenue_df.loc[budget_and_revenue_df.budget_millions == budget_and_revenue_df.budget_millions.max()]

,year,title,score,passing,imdb_id,director_gender,writer_gender,exec_gender,producer_gender,overall_gender,budget_millions,revenue_millions
21,2018.0,Avengers: Infinity War,3.0,1.0,tt4154756,0.0,0.0,1.0,0.0,1.0,400.0,678.815482


#### Movies with minimum and maximum revenues

In [31]:
budget_and_revenue_df.loc[budget_and_revenue_df.revenue_millions == budget_and_revenue_df.revenue_millions.min()]

,year,title,score,passing,imdb_id,director_gender,writer_gender,exec_gender,producer_gender,overall_gender,budget_millions,revenue_millions
7309,1935.0,A Night At The Opera,1.0,0.0,tt0026778,0.0,0.0,0.0,0.0,0.0,0.000001,0.000001


In [32]:
budget_and_revenue_df.loc[budget_and_revenue_df.revenue_millions == budget_and_revenue_df.revenue_millions.max()]

,year,title,score,passing,imdb_id,director_gender,writer_gender,exec_gender,producer_gender,overall_gender,budget_millions,revenue_millions
2556,2009.0,3 Idiots,1.0,0.0,tt1187043,0.0,0.0,0.0,0.0,0.0,350.0,3920.0


Observation: the min and max movies above demonstrate a problem with movie data, where there is not really one source for "ground truth" information. User submitted information can have errors, some websites may list revenue just in the US or revenue globally, financial data for international films may not be in US Dollars, older financial information may or may not be adjusted for inflation, and missing data may be represented in different ways. The budget and revenue for A Night at the Opera, for example, were probably not $1, and it is well known that Infinity War broke several box office records, with revenue in the billions - not millions.  

In [15]:
bechdel_df.groupby('passing').sum()

,year,score,director_gender,writer_gender,exec_gender,producer_gender,overall_gender,budget,revenue
passing,,,,,,,,,
0.0,6472828.0,3253.0,116.0,271.0,52.0,403.0,842.0,6.880390e+10,2.158132e+11
1.0,8714948.0,13083.0,616.0,980.0,114.0,760.0,2469.0,7.229692e+10,2.523760e+11
